## Keyword expansion

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from nltk.corpus import stopwords
import numpy as np
from nltk import word_tokenize, pos_tag, ne_chunk
from nltk import Tree
from nltk.stem import PorterStemmer 

# Begin with 1211 causes from news and literature
for country in countries:
    tf_cause = {}
    num_causes = 0
    file_type = 'cause'
    
    # These csv files are results from running the semantic frame parsing algorithm on news and literature
    df = pd.read_csv('results/{}_seeded_{}_cluster.csv'.format(country, file_type), delimiter=',')
    j = 1
    cause_thr = 0
    for cause, cause_sim in zip(df[df['{}_cluster'.format(file_type)] == i][file_type],
                                df[df['{}_cluster'.format(file_type)] == i]['cause_similarity']):
        num_causes += 1
        tokenized_doc  = word_tokenize(cause)
        tagged_sentences = pos_tag(tokenized_doc)
        stem_cause = " ".join([ps.stem(w) for w in cause.split(" ")])
        c_pre = ''
        for tag in tagged_sentences:
            c = ps.stem(tag[0])
            t = tag[1]
            if c in stemmed_stopwords or (not c.isalpha()) or c in black_list:
                continue
            if t.startswith('JJ'):
                c_pre = c
                continue
            if c_pre:
                c = c_pre + ' ' + c
                c_pre = ''
            if t.startswith('NN') and c_pre == '':
                c_pre = c
            tf_cause[c] = cause_sim[0]               
    cause_df = pd.DataFrame.from_dict(tf_cause, orient='index', columns=['count'])
    cause_df = cause_df.reset_index()
    causes_df = pd.concat([causes_df, cause_df], axis=0)

# Expanding 
for word, count in all_words.items():
    if count <= 1000:
        continue
    for cause in causes_df['cause'].unique():
        if glove_model.wmdistance(cause.split(), word.split()) < 6:
            tf_cause = {'cause': word, 'dist': glove_model.wmdistance(cause.split(), word.split())}
            cause_df = pd.DataFrame.from_dict(tf_cause)
            cause_df = cause_df.reset_index()
            causes_df = pd.concat([causes_df, cause_df])

causes_df.to_csv('expanded_cluster_causes.csv')

In [ ]:
# Get p-value of granger test for each of the variables with FEWS score
sign_ind = [i for i,v in enumerate(list(causes_df['p_value']<0.01)) if v]
causes = [causes_df.loc[i, 'cause'] for i in sign_ind]
X_filtered = [X[i] for i in sign_ind]